tôi đã sửa chatbot ban đầu ở dạng hiển thị  icon. 

vì vậy bạn hãy sửa lại login 1 chút. 

1. Ban đầu vào sẽ có chatbot embed luôn (và đương nhiên nó ở dạng icon). 

2. Sau khi click vào câu hỏi thì nó được bật lên và truyền Context và câu hỏi vô như cũ. 


# Test Cases



Hãy test các trường hợp sau để đảm bảo chatbot hoạt động đúng:

### 1. Khởi tạo ban đầu:
```typescript
Test case: Tải trang web
Kỳ vọng: 
- Icon chatbot (💬) hiển thị ở góc phải dưới
- Cửa sổ chat ẩn đi
```

### 2. Click vào icon:
```typescript
Test case: Click vào icon chatbot
Kỳ vọng:
- Icon biến mất
- Cửa sổ chat hiện ra
- Chat container hiển thị dạng flex
```

### 3. Click vào nút đóng (X):
```typescript
Test case: Click vào nút X trên cửa sổ chat
Kỳ vọng:
- Cửa sổ chat ẩn đi
- Icon chatbot hiện ra lại
- Lịch sử chat vẫn được giữ nguyên
```

### 4. Click vào câu hỏi (lần đầu):
```typescript
Test case: Click vào một câu hỏi trong RecommendQ
Kỳ vọng:
- Icon biến mất
- Cửa sổ chat hiện ra
- Tin nhắn (context + câu hỏi) hiển thị trong chat
- Bot trả lời câu hỏi
```

### 5. Click vào câu hỏi (sau khi đã đóng chat):
```typescript
Test case: 
1. Click câu hỏi
2. Đóng chat bằng nút X
3. Click câu hỏi khác
Kỳ vọng:
- Icon biến mất
- Cửa sổ chat hiện ra
- Lịch sử chat cũ vẫn còn
- Tin nhắn mới được thêm vào
```

### 6. Gửi nhiều câu hỏi liên tiếp:
```typescript
Test case: Click nhiều câu hỏi khác nhau
Kỳ vọng:
- Cửa sổ chat vẫn mở
- Các tin nhắn được thêm vào theo đúng thứ tự
- Không bị duplicate tin nhắn
```

### 7. Test tương tác song song:
```typescript
Test case: 
1. Nhập tin nhắn trực tiếp trong chat
2. Click câu hỏi từ RecommendQ
Kỳ vọng:
- Cả hai loại tin nhắn đều hiển thị đúng
- Thứ tự tin nhắn được giữ nguyên
```

### 8. Test khi mạng chậm:
```typescript
Test case: Giả lập mạng chậm và thực hiện các thao tác
Kỳ vọng:
- UI vẫn phản hồi mượt mà
- Tin nhắn vẫn được gửi và nhận đúng
- Không bị treo hoặc crash
```

### 9. Test đa thiết bị:
```typescript
Test case: Test trên desktop và mobile
Kỳ vọng:
- Layout hiển thị đúng trên mọi kích thước màn hình
- Icon và cửa sổ chat không bị che khuất
```

### 10. Test lỗi:
```typescript
Test case: Giả lập các tình huống lỗi
- Backend không phản hồi
- Script không tải được
- postMessage thất bại
Kỳ vọng:
- Hiển thị thông báo lỗi phù hợp
- Không crash ứng dụng
- Có thể tiếp tục sử dụng sau lỗi
```

Nếu phát hiện vấn đề trong bất kỳ test case nào, chúng ta sẽ cần điều chỉnh code để xử lý tình huống đó.



Tôi sẽ giúp bạn test từng case và đề xuất sửa đổi nếu cần:

### 1. Khởi tạo ban đầu ✅
Code hiện tại đã đúng:
````javascript
// embed.js
const toggleButton = document.createElement('button');
toggleButton.style.cssText = `
    // ...
    display: block; // Hiển thị mặc định
`;

const iframe = document.createElement('iframe');
iframe.style.cssText = `
    // ...
    display: none; // Ẩn mặc định
`;
````

### 2. Click vào icon ✅
Code đã xử lý đúng:
````javascript
toggleButton.addEventListener('click', () => {
    iframe.contentWindow.postMessage({ type: 'showChatbot' }, '*');
    iframe.style.display = 'block';
    toggleButton.style.display = 'none';
});
````

### 3. Click nút đóng (X) ✅
Code đã xử lý đúng:
````javascript
// script.js
document.getElementById('close-chatbot').addEventListener('click', () => {
    const chatContainer = document.querySelector('.chat-container');
    if (chatContainer) {
        chatContainer.style.display = 'none';
    }
    window.parent.postMessage({ type: 'closeChatbot' }, '*');
});
````

### 4 & 5. Click câu hỏi ⚠️
Cần sửa để đảm bảo tin nhắn được gửi sau khi chat đã mở:
````javascript
// embed.js
window.chatbot = {
    sendMessage: async (message) => {
        // Đảm bảo chat được mở trước
        iframe.style.display = 'block';
        toggleButton.style.display = 'none';
        
        // Đợi một chút để giao diện cập nhật
        await new Promise(resolve => setTimeout(resolve, 100));
        
        // Gửi tin nhắn
        iframe.contentWindow.postMessage({ type: 'sendMessage', message }, '*');
    }
};
````

### 6. Gửi nhiều câu hỏi ⚠️
Cần thêm xử lý chống duplicate:
````javascript
// script.js
let lastMessage = '';
let lastMessageTime = 0;

async function sendMessage(message) {
    // Chống duplicate trong 500ms
    const now = Date.now();
    if (message === lastMessage && now - lastMessageTime < 500) {
        return;
    }
    
    lastMessage = message;
    lastMessageTime = now;
    
    // Xử lý gửi tin nhắn như cũ
    ...
}
````

### 7. Tương tác song song ⚠️
Cần thêm queue xử lý tin nhắn:
````javascript
// script.js
const messageQueue = [];
let isProcessing = false;

async function processMessageQueue() {
    if (isProcessing || messageQueue.length === 0) return;
    
    isProcessing = true;
    const message = messageQueue.shift();
    
    try {
        await sendMessage(message);
    } finally {
        isProcessing = false;
        processMessageQueue();
    }
}

function queueMessage(message) {
    messageQueue.push(message);
    processMessageQueue();
}
````

### 8. Mạng chậm ⚠️
Thêm timeout và loading state:
````javascript
// script.js
async function sendMessage(message) {
    displayMessage(message, 'user');
    
    try {
        const controller = new AbortController();
        const timeoutId = setTimeout(() => controller.abort(), 10000);
        
        const response = await fetch(`${BACKEND_URL}/send_message`, {
            // ... các options khác ...
            signal: controller.signal
        });
        
        clearTimeout(timeoutId);
        // Xử lý response như cũ
    } catch (error) {
        if (error.name === 'AbortError') {
            displayMessage('Request timed out. Please try again.', 'error');
        } else {
            displayMessage('Error processing message.', 'error');
        }
    }
}
````

### 9. Đa thiết bị ⚠️
Thêm CSS responsive:
````css
@media (max-width: 768px) {
    #chatbot-iframe {
        width: 100% !important;
        height: 100% !important;
        right: 0 !important;
        bottom: 0 !important;
        border-radius: 0 !important;
    }
    
    #chatbot-toggle {
        bottom: 10px !important;
        right: 10px !important;
    }
}
````

### 10. Xử lý lỗi ⚠️
Thêm error boundary và retry logic:
````javascript
// embed.js
function initChatbot(options = {}) {
    try {
        // ... code khởi tạo ...
    } catch (error) {
        console.error('Chatbot initialization failed:', error);
        // Thử lại sau 5 giây
        setTimeout(() => initChatbot(options), 5000);
    }
}

// Thêm xử lý khi iframe load thất bại
iframe.onerror = () => {
    console.error('Failed to load chatbot iframe');
    toggleButton.style.display = 'block';
};
````

Những thay đổi này sẽ giúp chatbot:
1. Hoạt động ổn định hơn
2. Xử lý tốt các tình huống lỗi
3. Tương thích tốt trên nhiều thiết bị
4. Trải nghiệm người dùng mượt mà hơn

Bạn có thể áp dụng từng phần tùy theo mức độ ưu tiên của dự án.

# Sửa lỗi UI trắng khi click lần 2 

Okei cảm ơn bạn, đã hoạt động khá ngon. 
- Lần đầu tiên click vào câu hỏi thì chatbot đã tự động được bật lên, sau đó tự truyền vào câu trả lời và trả lời => OKE NGON
- Tuy nhiên sau đó tôi ấn dấu X để tắt chatbot. 
Click lại vào câu hỏi thì chatbot lại bật lên là cái UI trắng tinh (bị lỗi UI ý) (còn phía API thì vẫn thấy thông báo send message đã thành công và trả lời thành công) 

Tôi nghĩ có thể lỗi đến từ việc parent call tới chatbot lần đầu thì ngon, 
còn các lần sau call tới chatbot thì chưa ngon. 

Check xem vấn đề nằm ở đâu 

----
script.js, embed.js, hay @recommendQ.tsx  hay @ChatbotWidget.tsx 





Vấn đề có thể nằm ở cách xử lý hiển thị chat container trong `script.js`. Khi click vào câu hỏi lần thứ 2, iframe được hiển thị nhưng chat container bên trong không được set lại display flex.

### 1. Sửa `script.js`:

````javascript
// frontend/script.js
window.addEventListener('message', (event) => {
    // Khi nhận được message showChatbot hoặc sendMessage
    if (event.data?.type === 'showChatbot' || event.data?.type === 'sendMessage') {
        const chatContainer = document.querySelector('.chat-container');
        if (chatContainer) {
            chatContainer.style.display = 'flex';  // Luôn set display flex khi mở chat
        }
    }
    
    // Xử lý sendMessage riêng
    if (event.data?.type === 'sendMessage') {
        const message = event.data.message;
        sendMessage(message);
    }
});
````

### 2. Sửa `embed.js`:

````javascript
// frontend/embed.js
window.chatbot = {
    sendMessage: (message) => {
        iframe.contentWindow.postMessage({ type: 'showChatbot' }, '*'); // Thêm dòng này
        iframe.contentWindow.postMessage({ type: 'sendMessage', message }, '*');
        iframe.style.display = 'block';
        toggleButton.style.display = 'none';
    },
    showChat: () => {
        iframe.contentWindow.postMessage({ type: 'showChatbot' }, '*'); // Thêm dòng này
        iframe.style.display = 'block';
        toggleButton.style.display = 'none';
    }
};
````

Giải thích:
1. Trong `script.js`:
   - Thêm xử lý hiển thị chat container cho cả 2 event: `showChatbot` và `sendMessage`
   - Đảm bảo chat container luôn được set display flex khi mở chat

2. Trong `embed.js`:
   - Thêm việc gửi message `showChatbot` trước khi gửi tin nhắn
   - Đảm bảo chat container được hiển thị đúng trước khi xử lý tin nhắn

Với những thay đổi này:
1. Lần đầu click câu hỏi: Hoạt động bình thường
2. Sau khi đóng và click lại:
   - Iframe được hiển thị
   - Chat container được set display flex
   - Tin nhắn được gửi và hiển thị đúng

Bạn thử test lại xem có còn bị lỗi UI trắng tinh không nhé.